In [ ]:
#Connecting google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Getting the bert model
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2020-07-03 02:32:08--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 64.233.189.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  55.3MB/s    in 6.9s    

2020-07-03 02:32:15 (56.5 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
#importing dependencies
import pandas as pd
import numpy as np

In [ ]:
data_path= "/content/drive/My Drive/BERT/data/data.csv"

In [ ]:
data= pd.read_csv(data_path) #reading the data

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
!pip install bert-for-tf2 #dependencies

     |████████████████████████████████| 40kB 1.6MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=6378f52050c4cd25731cb6e398b0abd7da84f8725a455720e2f451fb41f86178
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=0b92fd4e3d3a3c8ae2509957ebfff17303983696df1c7a5c792c4933a9f09be7
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=81565358c6ebd5e6d7560b55c050b32a53090dbbbae5646bad75085360271db4
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [ ]:
!pip install sentencepiece #dependencies

     |████████████████████████████████| 1.1MB 2.6MB/s 


In [ ]:
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from bert.tokenization.bert_tokenization import FullTokenizer

In [ ]:
BERT_VOCAB= "/content/uncased_L-12_H-768_A-12/vocab.txt" #path to 'vocab.txt' file

In [ ]:
x= data["review"]
y= data["sentiment"]

In [ ]:
tokenizer= FullTokenizer(BERT_VOCAB) #initialization of tokenizer 

In [ ]:
# Getting stopwords to remove
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stop= stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Cleaning the data
x= x.str.replace(r'[^\w\s]', '')
x= x.apply(lambda x: " ".join(x.lower() for x in x.split() if x not in stop))

In [ ]:
x= x.apply(lambda x: tokenizer.tokenize(x)) #tokenization 

In [ ]:
x= x.apply(lambda x: ["[CLS]"]+x+["[SEP]"]) #Adding "[CLS]" and "[SEP]" at the end and beginning requirment of BERT

In [ ]:
x= x.apply(lambda x: tokenizer.convert_tokens_to_ids(x)) #Converting tokens to ID's

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
x= pad_sequences(x, padding= "post", maxlen= 200, truncating= "post") #padding the sequence

In [ ]:
import bert
from bert import BertModelLayer
import tensorflow as tf
import json
import os
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights

In [ ]:
# paths to BERT "config" and "ckpt" file
bert_config_file= "/content/uncased_L-12_H-768_A-12/bert_config.json"
max_seq_len= 200
bert_ckpt_file= "/content/uncased_L-12_H-768_A-12/bert_model.ckpt"

In [ ]:
# function for creating the model
def create_model(max_seq_len, bert_ckpt_file):
  with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert_params.adapter_size = None
      bert = BertModelLayer.from_params(bert_params, name="bert")
  input_ids = tf.keras.layers.Input(
    shape=(max_seq_len, ),
    dtype='int32',
    name="input_ids"
  )
  bert_output = bert(input_ids)
  print("bert shape", bert_output.shape)
  cls_out = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
  cls_out = tf.keras.layers.Dropout(0.5)(cls_out)
  logits = tf.keras.layers.Dense(units=768, activation="tanh")(cls_out)
  logits = tf.keras.layers.Dropout(0.5)(logits)
  logits = tf.keras.layers.Dense(
    units=2,
    activation="softmax"
  )(logits)
  model = tf.keras.Model(inputs=input_ids, outputs=logits)
  model.build(input_shape=(None, max_seq_len))
  load_stock_weights(bert, bert_ckpt_file)
  return model

In [ ]:
model= create_model(max_seq_len, bert_ckpt_file) # calling the model function

bert shape (None, 200, 768)
Done loading 196 BERT weights from: /content/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7f5a7aa8ac18> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [ ]:
from sklearn.preprocessing import label_binarize 

In [ ]:
y= label_binarize(y, classes=["positive", "negative"]) # Binarizing the tokens

In [ ]:
y

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 200)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 200, 768)          108890112 
_________________________________________________________________
lambda (Lambda)              (None, 768)               0         
_________________________________________________________________
dropout (Dropout)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 768)               590592    
_________________________________________________________________
dropout_1 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1538  

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(1e-5),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [ ]:
history = model.fit(
  x=x, y=y,
  validation_split=0.2,
  batch_size=16,
  shuffle=True,
  epochs=3)

Epoch 1/3
2500/2500 [==============================] - 3790s 2s/step - loss: 0.4703 - acc: 0.8337 - val_loss: 0.4365 - val_acc: 0.8723
Epoch 2/3
2500/2500 [==============================] - 3788s 2s/step - loss: 0.4192 - acc: 0.8905 - val_loss: 0.4310 - val_acc: 0.8799
Epoch 3/3
2316/2500 [==========================>...] - ETA: 4:19 - loss: 0.4027 - acc: 0.9073

In [ ]:
#model.save("/content/drive/My Drive/BERT/bearth_sentiment_analysis.h5")